**_This notebook is for exploring the data._**

In [1]:
import os
import time
import glob
from datetime import datetime

import numpy as np
import pandas as pd

import seaborn as sns
from pprint import pprint
import matplotlib.pyplot as plt

### Loading the Data

In [2]:
# Load the 2008 to May 2018 data
data1 = pd.read_csv("../data/SFPD_Crime_Data_2008_May_2018.csv")

# Load the 2018 to Present data
data2 = pd.read_csv("../data/SFPD_Crime_Data_2018_Present.csv")

# Check the shape of the data
print("The shape of the 2008 to May 2018 data is: {}".format(data1.shape))
print("The shape of the 2018 to Present data is: {}".format(data2.shape))

The shape of the 2008 to May 2018 data is: (2160953, 35)
The shape of the 2018 to Present data is: (394025, 36)


In [3]:
data1.head()

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,...,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
0,11087941463010,110879414,63010,WARRANTS,WARRANT ARREST,Friday,07/01/2011,08:00,NORTHERN,"ARREST, BOOKED",...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,17.0
1,5117770316010,51177703,16010,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,10/18/2005,14:30,TENDERLOIN,"ARREST, BOOKED",...,3.0,1.0,3.0,6.0,2.0,1.0,1.0,1.0,39.0,21.0
2,5011209707055,50112097,7055,VEHICLE THEFT,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Saturday,01/29/2005,13:45,BAYVIEW,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0
3,11044468164020,110444681,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,06/02/2011,02:52,CENTRAL,PSYCHOPATHIC CASE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,107.0
4,3038311107024,30383111,7024,VEHICLE THEFT,STOLEN TRAILER,Saturday,02/01/2003,08:00,BAYVIEW,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0


In [4]:
data2.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,HSOC Zones as of 2018-06-05,OWED Public Spaces,Central Market/Tenderloin Boundary Polygon - Updated,Parks Alliance CPSI (27+TL sites),ESNCAG - Boundary File,"Areas of Vulnerability, 2016"
0,2020/08/15 08:56:00 AM,2020/08/15,08:56,2020,Saturday,2020/08/15 08:56:00 AM,95300907041,953009,200474239,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020/08/15 09:40:00 AM,2020/08/15,09:40,2020,Saturday,2020/08/15 06:21:00 PM,95322706244,953227,206121692,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018/02/24 10:00:00 PM,2018/02/24,22:00,2018,Saturday,2018/03/02 10:13:00 AM,64174871000,641748,186051531,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020/08/16 03:13:00 AM,2020/08/16,03:13,2020,Sunday,2020/08/16 03:14:00 AM,95319604083,953196,200491669,202290313.0,...,54.0,2.0,9.0,26.0,NaN,NaN,NaN,NaN,NaN,2.0
4,2020/08/16 03:38:00 AM,2020/08/16,03:38,2020,Sunday,2020/08/16 04:56:00 AM,95326228100,953262,200491738,202290404.0,...,53.0,3.0,2.0,20.0,3.0,NaN,NaN,NaN,NaN,2.0


Let's consider which columns that we want to keep.
* Category
* Description (optional)
* Day of the Week
* Date
* Time
* District
* X coordinate
* Y coordinate

In [5]:
# Identify the subsets that we want to keep
subset1 = ["Category", "DayOfWeek", "Date", "Time", "PdDistrict", "X", "Y"]
subset2 = [
    "Incident Category",
    "Incident Day of Week",
    "Incident Date", 
    "Incident Time", 
    "Police District", 
    "Latitude", 
    "Longitude",
]

### Removing Null Values

In [6]:
# Identify the NULL values in the first database
data1.isnull().sum()

PdId                                                              0
IncidntNum                                                        0
Incident Code                                                     0
Category                                                          0
Descript                                                          0
DayOfWeek                                                         0
Date                                                              0
Time                                                              0
PdDistrict                                                        1
Resolution                                                        0
Address                                                           0
X                                                                 0
Y                                                                 0
location                                                          0
SF Find Neighborhoods 2 2                       

In [7]:
# Identify the NULL values in the second database
data2.isnull().sum()

Incident Datetime                                            0
Incident Date                                                0
Incident Time                                                0
Incident Year                                                0
Incident Day of Week                                         0
Report Datetime                                              0
Row ID                                                       0
Incident ID                                                  0
Incident Number                                              0
CAD Number                                               88433
Report Type Code                                             0
Report Type Description                                      0
Filed Online                                            312394
Incident Code                                                0
Incident Category                                          267
Incident Subcategory                                   

For the first dataset, it doesn't look like there are a lot of missing *important* data (i.e. date, time, district, category, etc.).

For the second dataset, it looks like there is a lot of missing *important* data. For example, there are 267 entries that are missing the category information, and there are over 20332 entries that are missing the latitude and longitude information.

Regardless, there is actually very little data loss since the dataset is so large.

In [8]:
# Remove the bad entries from dataset one.
data1.dropna(subset=subset1, inplace=True)

In [9]:
# Remove the bad entries from dataset two.
data2.dropna(subset=subset2, inplace=True)

In [10]:
# Confirm that there are no longer any NULL values for the desired subsets, for database 1.
data1.isnull().sum()

PdId                                                              0
IncidntNum                                                        0
Incident Code                                                     0
Category                                                          0
Descript                                                          0
DayOfWeek                                                         0
Date                                                              0
Time                                                              0
PdDistrict                                                        0
Resolution                                                        0
Address                                                           0
X                                                                 0
Y                                                                 0
location                                                          0
SF Find Neighborhoods 2 2                       

In [11]:
# Confirm that there are no longer any NULL values for the desired subsets, for database 2.
data2.isnull().sum()

Incident Datetime                                            0
Incident Date                                                0
Incident Time                                                0
Incident Year                                                0
Incident Day of Week                                         0
Report Datetime                                              0
Row ID                                                       0
Incident ID                                                  0
Incident Number                                              0
CAD Number                                               69527
Report Type Code                                             0
Report Type Description                                      0
Filed Online                                            306926
Incident Code                                                0
Incident Category                                            0
Incident Subcategory                                   

The data now contains no NULL values for the columns that we are interested in. Now we can move on to merging the dataset.

### Select Categories
* Create a new DataFrame that contains only the columns that we are interested in.
* To be consistent, the columns will be renamed and ordered as follows:
    * Category
    * Day of Week
    * Date
    * Time
    * District
    * Longitude
    * Latitude

In [12]:
master_columns = ["Category", "Day of Week", "Date", "Time", "District", "Longitude", "Latitude"]

In [19]:
# Select the subset from dataset one, and rename the columns if required
data1_subset = data1[subset1]

# Rename the categories
data1_subset.rename(columns={"DayOfWeek": "Incident Day of Week", "PdDistrict": "District", "X": "Longitude", "Y": "Latitude"}, inplace=True)

# Remove "Out of SF" from the District category if it exists in the first dataset.
data1_subset["District"] = data1_subset["District"][data1_subset["District"] != "Out of SF"]

data1_subset.head()

,Category,Incident Day of Week,Date,Time,District,Longitude,Latitude
0,WARRANTS,Friday,07/01/2011,08:00,NORTHERN,-122.439758,37.802151
1,DRUG/NARCOTIC,Tuesday,10/18/2005,14:30,TENDERLOIN,-122.414318,37.779944
2,VEHICLE THEFT,Saturday,01/29/2005,13:45,BAYVIEW,-122.388799,37.737576
3,NON-CRIMINAL,Thursday,06/02/2011,02:52,CENTRAL,-122.414354,37.803109
4,VEHICLE THEFT,Saturday,02/01/2003,08:00,BAYVIEW,-122.401097,37.724556


In [106]:
# Select the subset from dataset two.
data2_subset = data2[subset2]

# Rename the categories
data2_subset.rename(columns={"Incident Category" : "Category", "Incident Day of Week": "Day of Week", "Incident Date": "Date", "Incident Time": "Time", "Police District": "District"}, inplace=True)

# Swap the last two columns to be consistent with ordering.
data2_subset = data2_subset[master_columns]

data2_subset.head()

,Category,Day of Week,Date,Time,District,Longitude,Latitude
3,Assault,Sunday,2020/08/16,03:13,Bayview,-122.397729,37.754827
4,Malicious Mischief,Sunday,2020/08/16,03:38,Mission,-122.422044,37.766540
5,Non-Criminal,Sunday,2020/08/16,13:40,Southern,-122.403712,37.784044
6,Weapons Offense,Sunday,2020/08/16,16:18,Taraval,-122.507416,37.751003
7,Missing Person,Wednesday,2020/08/12,22:00,Northern,-122.432140,37.780496


### Merging Datasets
* Looks like there is some overlapping data. Let's examine the two columns and see how we can combine them. 

In [9]:
# Obtain the columns from both lists
columns1 = list(data1.columns)
columns2 = list(data2.columns)

pprint("Dataset 1's columns: {}".format(columns1))
pprint("Dataset 2's columns: {}".format(columns2))

("Dataset 1's columns: ['PdId', 'IncidntNum', 'Incident Code', 'Category', "
 "'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', "
 "'Address', 'X', 'Y', 'location', 'SF Find Neighborhoods 2 2', 'Current "
 "Police Districts 2 2', 'Current Supervisor Districts 2 2', 'Analysis "
 "Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2', 'DELETE - "
 "Police Districts 2 2', 'DELETE - Supervisor Districts 2 2', 'DELETE - Zip "
 "Codes 2 2', 'DELETE - Neighborhoods 2 2', 'DELETE - 2017 Fix It Zones 2 2', "
 "'Civic Center Harm Reduction Project Boundary 2 2', 'Fix It Zones as of "
 "2017-11-06  2 2', 'DELETE - HSOC Zones 2 2', 'Fix It Zones as of 2018-02-07 "
 "2 2', 'CBD, BID and GBD Boundaries as of 2017 2 2', 'Areas of Vulnerability, "
 "2016 2 2', 'Central Market/Tenderloin Boundary 2 2', 'Central "
 "Market/Tenderloin Boundary Polygon - Updated 2 2', 'HSOC Zones as of "
 "2018-06-05 2 2', 'OWED Public Spaces 2 2', 'Neighborhoods 2']")
("Dataset 2's columns: ['I

* The date and time information is consistent and can be merged.
* The "X" and "Y" location of the first dataset can be merged with the "Latitute" and "Longitude" information.
* The day-of-the-week information is in both datasets, so they can be merged easily.
* We can at least merge (for both datasets) the incident category. Looks like dataset1 is missing the subcategory information, so we either extrapolate or leave it out of the final combined dataset.
* It would also be easy to merge the Police District information, since they are present in both datasets.


#### Merge the Category Information
* Merge the category information, and reformat it into a consistent format.

In [84]:
# Determine the unique values for both datasets
data1_unique_values = list(data1_subset["Category"].unique())
data2_unique_values = list(data2_subset["Category"].unique())

pprint("Unique values for first dataset: {}".format(data1_unique_values))
pprint("Unique values for second dataset: {}".format(data2_unique_values))

("Unique values for first dataset: ['WARRANTS', 'DRUG/NARCOTIC', 'VEHICLE "
 "THEFT', 'NON-CRIMINAL', 'LARCENY/THEFT', 'BURGLARY', 'OTHER OFFENSES', "
 "'VANDALISM', 'PROSTITUTION', 'SUSPICIOUS OCC', 'DRUNKENNESS', 'ARSON', "
 "'EMBEZZLEMENT', 'FRAUD', 'ASSAULT', 'MISSING PERSON', 'ROBBERY', "
 "'FORGERY/COUNTERFEITING', 'SECONDARY CODES', 'SEX OFFENSES, FORCIBLE', "
 "'WEAPON LAWS', 'DISORDERLY CONDUCT', 'STOLEN PROPERTY', 'LOITERING', "
 "'SUICIDE', 'TRESPASS', 'LIQUOR LAWS', 'BAD CHECKS', 'KIDNAPPING', 'RECOVERED "
 "VEHICLE', 'DRIVING UNDER THE INFLUENCE', 'FAMILY OFFENSES', 'BRIBERY', "
 "'GAMBLING', 'SEX OFFENSES, NON FORCIBLE', 'EXTORTION', 'PORNOGRAPHY/OBSCENE "
 "MAT', 'TREA']")
("Unique values for second dataset: ['Assault', 'Malicious Mischief', "
 "'Non-Criminal', 'Weapons Offense', 'Missing Person', 'Larceny Theft', "
 "'Other', 'Lost Property', 'Burglary', 'Offences Against The Family And "
 "Children', 'Miscellaneous Investigation', 'Recovered Vehicle', 'Other "
 "Miscel

There are many inconsistencies.

For the 2018-Present dataset, there are more categories that are not present in the previous one. They are also written in a case-sensitive format. 

Without outside expertise, the best merge strategy here is just to combine the categories, after capitalizing all entries.

In [95]:
# Take the union of the two lists after making them all capitalized.
data2_category = data2_subset["Category"]
data2_category = data2_category.apply(lambda x: x.upper())

data2_category_unique = list(data2_category.unique())
master_category = list(set(data2_category_unique) | set(data1_unique_values))

50
70


#### Merge the Date Information
* Merge the date information, and reformat it into a consistent format. 
* The default format is (MM/DD/YYYY) for the date.

In [11]:
# Get the dates for both datasets
data1_date = data1["Date"]
data2_date = data2["Incident Date"]

print(data1_date.head())
print(data2_date.head())


0    07/01/2011
1    10/18/2005
2    01/29/2005
3    06/02/2011
4    02/01/2003
Name: Date, dtype: object
0    2020/08/15
1    2020/08/15
2    2018/02/24
3    2020/08/16
4    2020/08/16
Name: Incident Date, dtype: object


In [81]:
# For the dates in the second dataset, we are going to change the format.
def to_mmddyyyy(date):
    return datetime.strptime(date, "%Y/%m/%d").strftime("%m/%d/%Y")

data2_date_refactored = data2_date.apply(to_mmddyyyy)

print(data1_date.head())
print(data2_date_refactored.head())

0    07/01/2011
1    10/18/2005
2    01/29/2005
3    06/02/2011
4    02/01/2003
Name: Date, dtype: object
0    08/15/2020
1    08/15/2020
2    02/24/2018
3    08/16/2020
4    08/16/2020
Name: Incident Date, dtype: object


The dates are now in consistent formatting (MM/DD/YYYY) and can now be merged.

#### Merge the Time Information
* Merge the time information, and reformat it into a consistent format.
* The default format is (HH:MM) for the time, and follows the military time convention.

In [23]:
data1_time = data1["Time"]
data2_time = data2["Incident Time"]

print(data1_time.head())
print(data2_time.head())

0    08:00
1    14:30
2    13:45
3    02:52
4    08:00
Name: Time, dtype: object
0    08:56
1    09:40
2    22:00
3    03:13
4    03:38
Name: Incident Time, dtype: object


The time is already in consistent formatting and can thus be merged easily.

#### Merging the Day-Of-Week Information
* Merge the day-of-week information, and reformat it into a consistent format.
* The default format is the full name of the day (e.g. Monday, Tuesday, Wednesday, etc,) following the days of the week.

In [24]:
data1_dow = data1["DayOfWeek"]
data2_dow = data2["Incident Day of Week"]

print(data1_dow.head())
print(data2_dow.head())

0      Friday
1     Tuesday
2    Saturday
3    Thursday
4    Saturday
Name: DayOfWeek, dtype: object
0    Saturday
1    Saturday
2    Saturday
3      Sunday
4      Sunday
Name: Incident Day of Week, dtype: object


The day-of-week information is already in consistent formatting and can thus be merged easily.

#### Merging the District Information
* Merge the district information, and reformat it into a consistent format.
* The default format is the upper-case name of the district.

In [110]:
data1_district = data1_subset["District"]
data2_district = data2_subset["District"]

print(data1_district.unique())
print(data2_district.unique())

['NORTHERN' 'TENDERLOIN' 'BAYVIEW' 'CENTRAL' 'RICHMOND' 'MISSION'
 'SOUTHERN' 'TARAVAL' 'PARK' 'INGLESIDE']
['Bayview' 'Mission' 'Southern' 'Taraval' 'Northern' 'Central' 'Ingleside'
 'Park' 'Richmond' 'Out of SF' 'Tenderloin']


To merge the two sources, we are going to have to capitalize all entries in the District column for the second dataset, and then we are going to have to remove "Out of SF" entries. 

We can either remove this information before we do the merge, or we can remove the information during the merge.

#### Master Merge
* Now that we know how to make the information consistent, we are going to do an actual merge.

In [ ]:
# First, create the subsets.

In [112]:
# First, we deal with the category and district
data2_subset["Category"] = data2_subset["Category"].apply(lambda x: x.upper())
data2_subset["District"] = data2_subset["District"].apply(lambda x: x.upper())

data2_subset.head()

,Category,Day of Week,Date,Time,District,Longitude,Latitude
3,ASSAULT,Sunday,2020/08/16,03:13,BAYVIEW,-122.397729,37.754827
4,MALICIOUS MISCHIEF,Sunday,2020/08/16,03:38,MISSION,-122.422044,37.766540
5,NON-CRIMINAL,Sunday,2020/08/16,13:40,SOUTHERN,-122.403712,37.784044
6,WEAPONS OFFENSE,Sunday,2020/08/16,16:18,TARAVAL,-122.507416,37.751003
7,MISSING PERSON,Wednesday,2020/08/12,22:00,NORTHERN,-122.432140,37.780496
